In [6]:
import pandas as pd

# Read the data from the json file
movie = pd.read_json("new/movie_details_2.json")
movie.head()


,title,release_date,runtime,overview,budget,production_countries,revenue,genres,FILMID,backdrop_path,poster_path
0,Avatar,2009-12-10,162,"In the 22nd century, a paraplegic Marine is di...",237000000,US,2787965087,"[1, 2, 10, 21]",19995,/bIL7ENqh1egWTxN41sM2W42JqPc.jpg,/6EiRUJpuoeQPghrs3YNktfnqOVh.jpg
1,Pirates of the Caribbean: At World's End,2007-05-19,169,"Captain Barbossa, long believed to be dead, ha...",300000000,US,961000000,"[2, 10, 1]",285,/kobscH4qOsYQNTZnkrF5zkPCsvX.jpg,/jGWpG4YhpQwVmjyHEGkxEkeRf0S.jpg
2,Spectre,2015-10-26,148,A cryptic message from Bond’s past sends him o...,245000000,GB,880674609,"[1, 2, 6]",206647,/n1VZsWIi4missUgUXRfCG9HXFKe.jpg,/672kUEMtTHcaVYSVY4eiHEliHFa.jpg
3,The Dark Knight Rises,2012-07-16,165,Following the death of District Attorney Harve...,250000000,US,1084939099,"[1, 6, 8, 25]",49026,/yzsegNpkRCPadnWmshrJzT3ardR.jpg,/vzvKcPQ4o7TjWeGIn0aGC9FeVNu.jpg
4,John Carter,2012-03-07,132,"John Carter is a war-weary, former military ca...",260000000,US,284139100,"[1, 2, 21]",49529,/ubNevniKd5NYo60DqYIQ29T0uyB.jpg,/7GSSyUUgUEXm1rhmiPGSRuKoqnK.jpg


In [7]:
movie_crew = pd.read_excel("archive/CREW_CREDIT.xlsx")

In [8]:
movie_crew.head()

,ID,FILMID,CREDITID,DEPARTMENTID,PERSONID,JOBID,NAME
0,1,13053,52fe45339251416c7504efa7,8.0,7879.0,624.0,John Lasseter
1,2,13053,58ac8cc292514158df015f32,2.0,630.0,481.0,Mark Henley
2,3,13053,58ac843d92514158f6014ad1,12.0,2512.0,502.0,John W. Wheeler
3,4,13053,58ac8fb5c3a36849b00145df,7.0,8020.0,807.0,John Kahrs
4,5,13053,5626458ac3a3680e16014167,4.0,11098.0,756.0,John Powell


In [9]:
movie.drop_duplicates(subset ="FILMID", keep = False, inplace = True)

In [53]:
new_movie = movie[movie["release_date"] > "2000-01-01"]
new_movie = new_movie.reset_index(drop=True)
db_movie_director = movie_crew[movie_crew["FILMID"].isin(new_movie["FILMID"])]
db_movie_director = db_movie_director[db_movie_director["JOBID"] == 474]


In [54]:
db_movie_director = db_movie_director.drop_duplicates(subset ="FILMID", keep="first")
db_movie_director.drop(['ID', 'CREDITID', 'DEPARTMENTID', 'JOBID'], axis=1, inplace=True)
db_movie_director = db_movie_director.reset_index(drop=True)
db_movie_director.rename(columns = {'NAME':'director'}, inplace = True)
db_movie_director.head()

,FILMID,PERSONID,director
0,13053,70238.0,Chris Williams
1,100042,7395.0,Bobby Farrelly
2,537915,1586194.0,Jenny Gage
3,68724,82194.0,Neill Blomkamp
4,22894,116357.0,Scott Stewart


In [55]:
new_movie = pd.merge(new_movie, db_movie_director, on="FILMID", how="left")
with open("final/movies.json", "w") as f:
    f.write(new_movie.to_json(orient="records", lines=True))

In [60]:
director_list = db_movie_director[["PERSONID", "director"]]
director_list = director_list.drop_duplicates(subset ="PERSONID", keep="first")
director_list.shape

(1528, 2)

In [1]:
import pandas as pd
person = pd.read_excel("archive/PERSON.xlsx")
person.head()

,ID,PERSONID,NAME_,DEPARTMENTID,DEPARTMENT,BIRTHDAY,ALSO_KNOWN_AS,DEATHDAY,GENDER,BIOGRAPHY,PROFILE_PATH,POPULARITY,PLACE_OF_BIRTH,ADULT,IMDB_ID,HOMEPAGE,COUNTRYCODE,COUNTRY,COUNTRY2
0,1,984533,Adam Willis,3.0,Art,NaN,NaN,NaN,2,NaN,NaN,0.6,NaN,False,nm1853964,NaN,NaN,NaN,NaN
1,2,933593,Jodi Collins,8.0,Production,NaN,NaN,NaN,0,NaN,NaN,0.6,NaN,False,nm0172394,NaN,NaN,NaN,NaN
2,3,230274,John Kusiak,4.0,Sound,NaN,NaN,NaN,0,NaN,NaN,0.6,NaN,False,NaN,NaN,NaN,NaN,NaN
3,4,1662382,Robert Earl,8.0,Production,NaN,NaN,NaN,0,NaN,NaN,0.6,NaN,False,NaN,NaN,NaN,NaN,NaN
4,5,1667955,Pat Weber,5.0,Crew,NaN,NaN,NaN,0,NaN,NaN,0.6,NaN,False,NaN,NaN,NaN,NaN,NaN


In [62]:
director = pd.merge(director_list, person, on="PERSONID", how="left")
director = director[['PERSONID', 'NAME_', 'BIRTHDAY', 'DEATHDAY', 'BIOGRAPHY', 'PLACE_OF_BIRTH']]
director = director.reset_index(drop=True)
with open("final/directors.json", "w") as f:
    f.write(director.to_json(orient="records", lines=True))

In [3]:
import pandas as pd
movie_actor = pd.read_json("final/movie_actor.json")
movie_actor.shape

(119702, 4)

In [6]:
actor = movie_actor[["actor_id", "actor_name"]]
actor = actor.drop_duplicates(subset ="actor_id", keep="first")
with open("final/actors.json", "w") as f:
    f.write(actor.to_json(orient="records", lines=True))

In [2]:
actor = pd.read_json("final/actors.json")
actor.shape

(67649, 3)

In [11]:
actor_data = pd.merge(actor, person[["PERSONID","BIRTHDAY", "DEATHDAY", "PLACE_OF_BIRTH", "BIOGRAPHY", "PROFILE_PATH"]], left_on="actor_id", right_on="PERSONID", how="left")

In [12]:
actor_data.drop_duplicates(subset ="actor_id", keep="first", inplace=True)
actor_data.shape

(67649, 9)

In [13]:
actor_data.head()
with open("final/actor_images_2.json", "w") as f:
    f.write(actor_data.to_json(orient="records", lines=True))